In [ ]:
# https://medium.com/geekculture/step-by-step-decision-tree-id3-algorithm-from-scratch-in-python-no-fancy-library-4822bbfdd88f
# ID3

Attribute vs Measure
https://analystanswers.com/what-is-a-data-attribute-definition-types-examples/

Entropy quantifies how much information there is in a random variable, or more specifically its probability distribution. A skewed distribution has a low entropy, whereas a distribution where events have equal probability has a larger entropy.

In [2]:
import pandas as pd 
import numpy as np 

In [5]:
train_data_m = pd.read_csv("../datafiles/PlayTennis.xls")

In [6]:
train_data_m.head()

,Outlook,Temperature,Humidity,Wind,Play Tennis
0,Sunny,Hot,High,Weak,No
1,Sunny,Hot,High,Strong,No
2,Overcast,Hot,High,Weak,Yes
3,Rain,Mild,High,Weak,Yes
4,Rain,Cool,Normal,Weak,Yes


In [7]:
train_data_m["Outlook"].unique()

array(['Sunny', 'Overcast', 'Rain'], dtype=object)

In [8]:
train_data_m[train_data_m["Outlook"] == "Overcast"]

,Outlook,Temperature,Humidity,Wind,Play Tennis
2,Overcast,Hot,High,Weak,Yes
6,Overcast,Cool,Normal,Strong,Yes
11,Overcast,Mild,High,Strong,Yes
12,Overcast,Hot,Normal,Weak,Yes


GainRatio is used to select the node splitting attribute and the splitting point of the continuous attribute value in the C4.5 algorithm

A_i is an attribute in the attribute set A to be split, a_1 and a_2 are weight coefficients assigned to the information gain and the gainratio: 0 <= a_1, a_2 <= 1, and a_1 + a_2 = 1.

IG_GR(Data, A_i) = (a_1 + a_2 / SplitInformation(D, A_i)) * InfoGain(D, A_i)

##### C4.5 
https://github.com/barisesmer/C4.5/tree/master/c45

https://github.com/Valdecy/C4.5/blob/master/Python-DM-Classification-04-C4.5.py

https://github.com/loginaway/DecisionTree

In [69]:
import numpy as np

class Node(object):
    '''
    Basic element for decision tree.
    Initialization: to initialize a tree node, at least feature_name is needed.
    Connection: to connect different nodes, there are two situations.
        1, if the parent node is discrete (i.e. node.discrete==True), 
            then 
            >> node[feature_value]=childnode 
            will do the connection.
        2, if the parent node is continuous, then 
            >> node['<=']=childnode 
            or 
            >> node['>']=childnode 
            will assign its children. To be specific, 
            node['<=']=childnode means if the feature value is bigger than 
            node.threshold, then the workflow goes left to the childnode, and vice versa.
    
    Classification: if you have constructed a tree, you may want to classify a 
        certain group of data. Simply call the root node will do this task, i.e.
        >> root(data)
        where data should be a dictionary containing the corresponding key and value.
        And this method will return the classification result.
    Visualization: A naive visualization is implemented here. 
        For example, you may visualize a subtree rooted at 'node' by
        >> print(node)
        The tree structure will then be printed on the console.
    '''
    def __init__(self, feature_name='', discrete=True, threshold=0, isLeaf=False, classification=None):
        # for discrete node: the next node can be retrieved by self.map[feature_value]
        # for continuous node: by self.map['<='] (when value <= self.threshold)
        #                       by self.map['>'] (when value > self.threshold)
        self.map=dict()

        self.discrete=discrete
        self.feature_name=feature_name
        self.isLeaf=isLeaf
        if isLeaf:
            self.classification=classification
        if not discrete:
            self.threshold=threshold

    def __setitem__(self, key, value):
        self.map[key]=value

    def __getitem__(self, key):
        return self.map.get(key)

    def __call__(self, data):
        '''
        This method should be used on the root to predict its classification.
        data: a dict with its key being the features and value being 
        the corresponding value.
        '''

        #print("Inside __call__ method of Node")

        #print("data: ")
        #print(data)

        #print("feature name: ")
        #print(self.feature_name)

        #print("the map right now")
        #print(self.map)

        if self.isLeaf:
            return self.classification
        # if the node has a discrete feature, use __getitem__ to find the next node
        # then call the next node.
        if self.discrete:
            # print("feature was discrete!")
            
            return self.map[data[self.feature_name]](data)
        else:
        # node is not discrete: print the self.feature_name
            #print("continuous name")
            #print(self.feature_name)
            if data[self.feature_name].astype(np.float32)>self.threshold:
                return self.map['>'](data)
            else:
                return self.map['<='](data)

    def __str__(self):
        '''
        This method is designed for printing the subtree rooted at the current node.
        To print the tree, try print(self).
        '''
        hierarchy={}
        stack=[(0, 0, self)]
        count=0
        while stack:
            # BFS
            (layer_index, node_index, current)=stack.pop(0)
            layer_index+=1

            hierarchy[layer_index]=hierarchy.get(layer_index, [])
            hierarchy[layer_index].append((node_index, current.feature_name,\
                    [(str(current.classification),)] if current.isLeaf else \
                     [(str(key), '' if current.discrete else str(current.threshold), item.feature_name) for key, item in current.map.items()]))
            # print(hierarchy)

            if current.isLeaf:
                continue
            for _, item in current.map.items():
                count+=1
                stack.append((layer_index, count, item))
            # print(item_index_map)

        totallist=[]
        for layer, layer_content in hierarchy.items():
            rowlist=[]
            for i in layer_content:
                rowlist.append('['+i[1]+']'+': {'+', '.join([' '.join(item) for item in i[2]])+'}')
            rowstr=' +++ '.join(rowlist)
            totallist.append(rowstr)

        return '============Root===========\n'+'\n\n'.join(totallist)+'\n============Leaf==========='

Today: the tree "works", or some approximation of it? and With Adult Data. 

In [105]:
#D6= np.genfromtxt('../datafiles/iris_train.txt', dtype=str)
D6 = adult23.to_numpy()
D6 = D6.astype(str)
np.random.shuffle(D6)

k=len(D6)

#attribute_name=['sepal length', 'sepal width', 'petal length', 'petal width']

attribute_name = ["Age", "Education", "Occupation", "Gender"]

A={name: [i] for i, name in enumerate(attribute_name)}

print(A)

# feature_discrete = {name: False for name in attribute_name}
feature_discrete={name: False if isinstance(adult2[name].iloc[0], np.integer) else True for name in attribute_name}

conf={'A': A, 
      'feature_discrete': feature_discrete, 
      'treeType':'IGGR'}


dt=Tree_OLD(conf)

dt.train(D6[:len(D6)//2])
print(dt.tree)
print(dt.eval(D6[len(D6)//2:]))
dt.prune(D6[:len(D6)//2], D6[len(D6)//2:])
print(dt.tree)
print(dt.eval(D6[len(D6)//2:]))

{'Age': [0], 'Education': [1], 'Occupation': [2], 'Gender': [3]}
============Root===========
[Age]: {<= 23.5 leaf-0, > 23.5 Age}

[leaf-0]: {1} +++ [Age]: {<= 71.0 Education, > 71.0 leaf-386}

[Education]: {0  Age, 1  Occupation, 10  Age, 11  Age, 12  Age, 14  Occupation, 15  Gender, 2  Occupation, 3  leaf-243, 4  Age, 5  Age, 6  Occupation, 7  Gender, 8  Age, 9  Age} +++ [leaf-386]: {1}

[Age]: {<= 52.0 leaf-1, > 52.0 Age} +++ [Occupation]: {-1  leaf-4, 0  leaf-5, 10  leaf-6, 11  leaf-7, 12  leaf-8, 13  leaf-9, 2  Age, 3  leaf-12, 4  leaf-13, 5  Age, 6  leaf-17, 7  leaf-18, 8  leaf-19, 9  leaf-20} +++ [Age]: {<= 36.5 leaf-21, > 36.5 Gender} +++ [Age]: {<= 25.5 leaf-25, > 25.5 Age} +++ [Age]: {<= 33.5 leaf-113, > 33.5 Age} +++ [Occupation]: {-1  leaf-154, 0  leaf-155, 10  leaf-156, 11  leaf-157, 12  leaf-158, 13  leaf-159, 2  leaf-160, 3  leaf-161, 4  leaf-162, 5  leaf-163, 6  leaf-164, 7  leaf-165, 8  leaf-166, 9  Age} +++ [Gender]: {0  Age, 1  Age} +++ [Occupation]: {-1  leaf-229, 0 

In [ ]:
D6

In [ ]:
A

In [341]:
len(D2[:, 4])

1628

In [347]:
herlist = D2[:, 4]
unique, counts = np.unique(herlist, return_counts=True)
print(np.asarray((unique, counts)).T)

[['0' '1554']
 ['1504' '1']
 ['1590' '1']
 ['1594' '1']
 ['1602' '1']
 ['1617' '1']
 ['1628' '1']
 ['1651' '1']
 ['1672' '1']
 ['1719' '1']
 ['1721' '1']
 ['1741' '1']
 ['1762' '1']
 ['1848' '6']
 ['1876' '2']
 ['1887' '10']
 ['1902' '9']
 ['1974' '1']
 ['1977' '13']
 ['1980' '2']
 ['2001' '1']
 ['2002' '2']
 ['2051' '1']
 ['2057' '1']
 ['2080' '1']
 ['2179' '1']
 ['2206' '1']
 ['2246' '1']
 ['2258' '1']
 ['2267' '1']
 ['2339' '1']
 ['2377' '1']
 ['2415' '3']
 ['2444' '1']
 ['2559' '2']]


In [354]:
print(float('2559') > float('915.0'))

True


In [356]:
np.unique(D2[:, 4].astype(float)> float(str(951.0)))

array([False,  True])

In [357]:
D2[np.argwhere(D2[:, 4].astype(float) > 951.0), -1].flatten()

array(['0', '1', '1', '1', '0', '0', '1', '0', '0', '1', '0', '0', '0',
       '1', '1', '1', '1', '1', '0', '0', '1', '1', '0', '0', '0', '0',
       '1', '0', '0', '1', '0', '1', '0', '0', '0', '0', '0', '0', '1',
       '0', '1', '1', '0', '1', '0', '0', '0', '0', '0', '1', '1', '1',
       '0', '1', '0', '0', '0', '0', '1', '1', '0', '0', '1', '0', '0',
       '0', '1', '0', '0', '0', '0', '0', '0', '0'], dtype='<U21')

In [71]:
import sys
import numpy as np
import math
#from Node import Node

# set the maximal recursion limits here.
sys.setrecursionlimit(10000)

# to be modified to single class

# attribute_indexes, - A_ind
# attribute_values, A
# root, 
# class_values, 
# feature_discrete
# treetype 
# dataset_size, 
# epsilon_per_tree


class Tree_DPDT(DPRF_Forest):

    '''
    The main class of decision tree.
    '''
    def __init__(self, A_ind, A, attribute_values, root, class_values, feature_discrete, treetype, dataset_size, epsilon_per_tree):
        '''
        attribute_values : attribute_values
        A : orignal
        feature_discrete: a dict with its each key-value pair being (feature_name: True/False),
            where True means the feature is discrete and False means the feature is 
            continuous. 
        type: ID3/C4.5/CART
        pruning: pre/post
        '''

        self.A=A
        self.attribute_values=attribute_values
        self.A_ind=A_ind
        self.feature_discrete= feature_discrete
        self.treeType=treetype
        self.leaf_count=0
        self.tmp_classification=''
        self.class_values=class_values
        self._root_node = root
        self.tree=None
        self.dataset_size=dataset_size
        self.epsilon=epsilon_per_tree

    def Entropy(self, list_of_class):
        '''
        Compute the entropy for the given list of class.
        list_of_class: an array of classification labels, e.g. ['duck', 'duck', 'dolphin']
        'duck': 2/3, 'dolphin': 1/3, so the entropy for this array is 0.918
        '''
        count={}
        for key in list_of_class:
            count[key]=count.get(key, 0)+1
        frequency=np.array(tuple(count.values()))/len(list_of_class)
        return -1*np.vdot(frequency, np.log2(frequency))

    def Information_Gain(self, list_of_class, grouped_list_of_class):
        '''
        Compute the Information Gain.
        list_of_class: an array of classification labels, e.g. ['duck', 'duck', 'dolphin']
        grouped_list_of_class: the list of class grouped by the values of 
            a certain attribute, e.g. [('duck'), ('duck', 'dolphin')].
        The Information_Gain for this example is 0.2516.
        '''
        sec2=np.sum([len(item)*self.Entropy(item) for item in grouped_list_of_class])/len(list_of_class)
        return self.Entropy(list_of_class)-sec2


    def Information_Ratio(self, list_of_class, grouped_list_of_class):
        '''
        Compute the Information Ratio.
        list_of_class: an array of classification labels, e.g. ['duck', 'duck', 'dolphin']
        grouped_list_of_class: the list of class grouped by the values of 
            a certain attribute, e.g. [('duck'), ('duck', 'dolphin')].
        The Information_Ratio for this example is 0.2740.
        '''
        tmp=np.array([len(item)/len(list_of_class) for item in grouped_list_of_class])
        
        # Here we assume instrinsic_value is SplitInformation! 
        intrinsic_value=-1*np.vdot(tmp, np.log2(tmp))

        return self.Information_Gain(list_of_class, grouped_list_of_class) / intrinsic_value

    def IGGR(self, list_of_class, grouped_list_of_class, a1, a2):

        sec2 = np.sum([len(item)*self.Entropy(item) for item in grouped_list_of_class])/len(list_of_class)
        
        infoGain =  self.Entropy(list_of_class)-sec2

        #print("IGGR grouped_list_of_class")
        #print("----------")
        #print(grouped_list_of_class)
        #print("----------")

        tmp = np.array([len(item)/len(list_of_class) for item in grouped_list_of_class])
        
        intrinsic_value = -1*np.vdot(tmp, np.log2(tmp))

        GainRatio = self.Information_Gain(list_of_class, grouped_list_of_class)/ intrinsic_value

        sol1 = a1 * infoGain + a2 * GainRatio
        sol2 = (a1 + ( a2 / intrinsic_value) ) * infoGain 

        return sol2

    def orderByGainOrRatio(self, D, A, by='Gain'):
        '''
        Return the order by Information Gain or Information Ratio.
        by: 'Gain', 'Ratio'.
        For the definition of D and A, see the remark in method 'fit'.
        '''
        tmp_value_dict=dict()

        target_function = self.Information_Gain if by=='Gain' else self.Information_Ratio

        for attr, info in A.items():
            possibleVal=np.unique(D[:, info[0]])
            # if the continuous attribute have only one possible value, then 
            # choosing it won't improve the model, so we abandon it.
            if len(possibleVal)==1:
                continue

            if self.feature_discrete[attr] is True:
                # discrete
                if len(info)<2:
                    A[attr].append(possibleVal)
                # retrieve the grouped list of class
                grouped_list_of_class=[]

                for val in possibleVal:

                    indexes=np.argwhere(D[:, info[0]]==val)
                    grouped_list_of_class.append(D[indexes, -1].flatten())

                IC_value=target_function(D[:, -1], grouped_list_of_class)
                tmp_value_dict[attr]=IC_value

            else:

                # continuous
                split_points=(possibleVal[: -1].astype(np.float32)+possibleVal[1:].astype(np.float32))/2
                maxMetric=-1

                for point in split_points:
                    smaller_set=D[np.argwhere(D[:, info[0]]<=str(point)), -1].flatten()
                    bigger_set=D[np.argwhere(D[:, info[0]]>str(point)), -1].flatten()

                    # compute the metric
                    IC_tmp=target_function(D[:, -1], (smaller_set, bigger_set))
                    if IC_tmp>maxMetric:
                        maxMetric=IC_tmp
                        threshold=point

                # set the threshold
                if len(info)<2:
                    A[attr].append(threshold)
                else:
                    A[attr][1]=threshold
                tmp_value_dict[attr]=maxMetric

        # find the attribute with the max tmp_value_dict value
        attr_list=list(tmp_value_dict.keys())
        attr_list.sort(key=lambda x: tmp_value_dict[x])

        return attr_list

    def orderByIGGR(self, D, A, a1, a2):
        '''
        Return the order by Information Gain or Information Ratio.
        by: 'Gain', 'Ratio'.
        For the definition of D and A, see the remark in method 'fit'.
        '''
        tmp_value_dict=dict()

        target_function1 = self.Information_Gain
        target_function2 = self.Information_Ratio
        target_function3 = self.IGGR
        
        for attr, info in A.items():
            possibleVal=np.unique(D[:, info[0]])
            # if the continuous attribute have only one possible value, then 
            # choosing it won't improve the model, so we abandon it.
            if len(possibleVal)==1:
                continue

            if self.feature_discrete[attr] is True:
                # discrete
                if len(info)<2:
                    A[attr].append(possibleVal)
                # retrieve the grouped list of class
                grouped_list_of_class=[]
                for val in possibleVal:
                    indexes=np.argwhere(D[:, info[0]]==val)
                    grouped_list_of_class.append(D[indexes, -1].flatten())
                IC_value=target_function3(D[:, -1], grouped_list_of_class, a1, a2)
                tmp_value_dict[attr]=IC_value

            else:
                # continuous


                split_points=(possibleVal[: -1].astype(np.float32)+possibleVal[1:].astype(np.float32))/2
                maxMetric=-1
                for point in split_points:

                    # modified now to float comparisons
                    # causes the code to crash to max recursion
                    # smaller_set = D[np.argwhere(D[:, info[0]].astype(float) <= point ), -1].flatten()
                    # info[0] = 4, point = 951.0
                    # bigger_set = D[np.argwhere(D[:, info[0]].astype(float) > point ), -1].flatten()
                    
                    smaller_set = D[np.argwhere(D[:, info[0]] <= str(point) ), -1].flatten()
                    # info[0] = 4, point = 951.0
                    bigger_set = D[np.argwhere(D[:, info[0]] > str(point) ), -1].flatten()

                    # compute the metric
                    # list_of_class, grouped_list_of_class, a1, a2
                    IC_tmp=target_function3(D[:, -1], (smaller_set, bigger_set), a1, a2)

                    if IC_tmp>maxMetric:
                        maxMetric=IC_tmp
                        threshold=point

                    # to stop the threshold from breaking the code
                    
                    #print("the inputs to bigger set producing empty list")
                    #print(info[0], point)

                    #print("attribute was: ", attr)
                    #print("sets (small and big)")
                    #print(smaller_set)
                    #print(bigger_set)

                    # IC_tmp=0.0

                # set the threshold
                if len(info)<2:
                    A[attr].append(threshold)
                else:
                    A[attr][1]=threshold
                tmp_value_dict[attr]=maxMetric

        # find the attribute with the max tmp_value_dict value
        attr_list=list(tmp_value_dict.keys())
        attr_list.sort(key=lambda x: tmp_value_dict[x])

        return attr_list

    def chooseAttribute(self, D, A):

        '''
        Choose an attribute from A according to the metrics above.
        For the definition of D and A, see method 'fit'.
        Different principal for different tree types:
        ID3: choose the attribute that maximizes the Information Gain.
        C4.5: 
            1, choose those attributes whose Information Gain are above average.
            2, choose the one that maximizes the Gain Ratio from these attributes.
        CART: choose the attribute that minimizes the Gini Index.
        IG_GR: 
        '''
        if self.treeType=='ID3':
            attr_list=self.orderByGainOrRatio(D, A, by='Gain')
            return attr_list[-1]

        if self.treeType=='C4.5':
            attr_list=self.orderByGainOrRatio(D, A, by='Gain')

            # for C4.5, we choose the attributes whose Gain are above average
            # and then order them by Ratio.

            sub_A={key: A[key] for key in attr_list}
            attr_list=self.orderByGainOrRatio(D, sub_A, by='Ratio')

            return attr_list[-1]

        if self.treeType=='IGGR':

            attr_list=self.orderByIGGR(D, A, 0.4, 0.6)

            return attr_list[-1]

    def train(self, D):
        self.tree=self.fit(D, self.A)

    def fit(self, D, A):

        '''
        Train the tree.
        To save the training result:
        >> self.tree=self.fit(D, A)
        D: the training set, a size [m, n+1] numpy array (with str type elements), 
            where m is the number of training data and n is the number of attributes.
            The last column of D is the classifications (or labels).
        A: the attributes set. It is a dict with its structure being like 
            {attr_name: [index_in_D_columns, possibleVal_or_threshold], ...}
            attr_name: name of the attribute
            index_in_D_columns: the corresponding index of the attribute in ndarray D (starting from 0)
            possibleVal_or_threshold: 
                ###################################################
                ## This value may not always be available in A   ##
                ## it is added after 'chooseAttribute' is called ##
                ## And it will be updated after each call        ##
                ###################################################
                1, if the attribute is discrete, then it is a ndarray containing all possible values 
                    of this attribute.
                2, if the attribute is continuous, then possibleVal_or_threshold is the most recent 
                    threshold.
        '''

        # termination conditions 

        # the training set is empty
        if len(D)==0:
            node=Node(feature_name='leaf-'+str(self.leaf_count), isLeaf=True, \
                classification=self.tmp_classification)
            self.leaf_count+=1
            return node

        # only one type of classification is left 
        if len(np.unique(D[:, -1]))<=1:
            node=Node(feature_name='leaf-'+str(self.leaf_count), isLeaf=True, \
                classification=D[0, -1])
            self.leaf_count+=1
            return node

        if len(A)==0 or len(np.unique(D[:, :-1], axis=0))<=1:
            count_dict={}
            for key in D[:, -1]:
                count_dict[key]=count_dict.get(key, 0)+1
            most_frequent=sorted(D[:, -1], key=lambda x: count_dict[x])[-1]

            node=Node(feature_name='leaf-'+str(self.leaf_count), isLeaf=True, \
                classification=most_frequent)
            self.leaf_count+=1
            return node 

        count_dict={}
        for key in D[:, -1]:
            count_dict[key]=count_dict.get(key, 0)+1
        most_frequent=sorted(D[:, -1], key=lambda x: count_dict[x])[-1]
        self.tmp_classification=most_frequent

        # choose target attribute
        target_attr=self.chooseAttribute(D, A)
        # print(target_attr)

        # generate nodes for each possible values of the target attribute if it's discrete
        # generate two nodes for the two classification if it's continuous
        # related information is stored in A[target_attr][1] now, 
        # since we have called chooseAttribute at least once.
        info=A[target_attr]

        if self.feature_discrete[target_attr]:

            node=Node(feature_name=target_attr, discrete=True, isLeaf=False)
            # generate nodes for each possible values
            
            for possibleVal in info[1]:
                keys=set(A.keys()).difference({target_attr})
                # connect node to its child
                tmp_D=D[np.argwhere(D[:, info[0]]==possibleVal), :]
                tmp_A={key: A[key] for key in keys}

                node[possibleVal]=self.fit(tmp_D.reshape((tmp_D.shape[0], tmp_D.shape[2])), tmp_A)
        
        else:

            # continuous
            threshold=info[1]
            node=Node(feature_name=target_attr, discrete=False, threshold=threshold, isLeaf=False)

            tmp_D=D[np.argwhere(D[:, info[0]]<=str(threshold)), :]
            node['<=']=self.fit(tmp_D.reshape((tmp_D.shape[0], tmp_D.shape[2])), A)

            tmp_D=D[np.argwhere(D[:, info[0]]>str(threshold)), :]
            node['>']=self.fit(tmp_D.reshape((tmp_D.shape[0], tmp_D.shape[2])), A)
            
        
        return node

    def prune(self, training_D, testing_D):
        print("We got to the pruning stage!")
        self.post_prune(training_D, testing_D, self.A, current=self.tree)

    def post_prune(self, training_D, testing_D, A, current=None, parent=None):
    
        '''
        self.tree is required.
        This method conducts the post-pruning to enhance the model performance.
        To make sure this method will work, set 
        >> current=self.tree
        when you call it.
        '''

        # print("prune prints!")

        self.current_accuracy=self.evaluate(testing_D, A)

        count_dict={}
        if len(training_D)==0:
            return 

        for key in training_D[:, -1]:
            count_dict[key] = count_dict.get(key, 0)+1
        most_frequent=sorted(training_D[:, -1], key=lambda x: count_dict[x])[-1]

        leaf_parent=True
        for key, node in current.map.items():
            if not node.isLeaf:
                leaf_parent=False
                # Recursion, DFS
                if node.discrete:
                    tmp_D=training_D[np.argwhere(training_D[:, A[current.feature_name][0]]==key), :]
                else:
                    if key=='<=':
                        tmp_D=training_D[np.argwhere(training_D[:, A[current.feature_name][0]]<=str(node.threshold)), :]
                    else:
                        tmp_D=training_D[np.argwhere(training_D[:, A[current.feature_name][0]]>str(node.threshold)), :]
                self.post_prune(tmp_D.reshape((tmp_D.shape[0], tmp_D.shape[2])), testing_D, A, parent=current, current=node)
        
        tmp_node=Node(feature_name='leaf-'+str(self.leaf_count), isLeaf=True, classification=most_frequent)
        
        if parent:
            # when current node is not the root
            for key, node in parent.map.items():
                if node==current:
                    parent.map[key]=tmp_node
                    saved_key=key
                    break
            # compare the evaluation, if it is enhanced then prune the tree.
            tmp_accuracy=self.evaluate(testing_D, A)
            if tmp_accuracy<self.current_accuracy:
                parent.map[saved_key]=current
                
            else:
                self.current_accuracy=tmp_accuracy
                self.leaf_count+=1
            return
        else:
            # when current node is the root
            saved_tree=self.tree
            self.tree=tmp_node
            tmp_accuracy=self.evaluate(testing_D, A)
            if tmp_accuracy<self.current_accuracy:
                self.tree=saved_tree
            else:
                self.current_accuracy=tmp_accuracy
                self.leaf_count+=1
            return

    def pred(self, D):
        return self.predict(D, self.A)
    
    def eval(self, D):
        return self.evaluate(D, self.A)

    def predict(self, D, A):
        '''
        Predict the classification for the data in D.
        For the definition of A, see method 'fit'. 
        There is one critical difference between D and that defined in 'fit':
            the last column may or may not be the labels. 
            This method works as long as the feature index in A matches the corresponding
            column in D.
        '''
        
        print("tree")
        print(self.tree) 

        row, _= D.shape # for the entire testing data 
        pred=np.empty((row, 1), dtype=str)
        tmp_data={key: None for key in A.keys()}

        #print("tmp_data")
        #print(tmp_data)
        #print("------")
        #print("inside of A ")
        #print(A)

        # IndexError: only integers, slices (`:`), ellipsis (`...`), 
        # numpy.newaxis (`None`) and integer or boolean arrays are valid indices

        # print("keys that do not agree with tmp_data: ")
        for i in range(len(D)):
            for key, info in A.items():

                tmp_data[key] = D[i, info[0]]

            pred[i]=self.tree(tmp_data)
            
        return pred
    
    def evaluate(self, testing_D, A):
        '''
        Evaluate the performance of decision tree. (Accuracy)
        For definition of testing_D and A, see 'predict'.
        However, here the testing_D is required to be labelled, that is, its last column 
        should be labels of the data.
        '''
        true_label=testing_D[:, -1]
        pred_label=self.predict(testing_D, A)
        
        success_count=0
        for i in range(len(true_label)):
            if true_label[i]==pred_label[i]:
                success_count+=1

        return success_count/len(true_label)


In [8]:
import pandas as pd 
import numpy as np 

dataset = 'https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data'
row_names = ["Age", "Workclass", "Fnlwgt", "Education", "EducationNum", "MaritalStatus",
        "Occupation", "Relationship", "Race", "Gender", "CapitalGain", "CapitalLoss",
        "HoursPerWeek", "Country", "Income"]
us_adult_income = pd.read_csv(dataset, names=row_names,na_values=[' ?'])
us_adult_income["Income"] = pd.Categorical(us_adult_income["Income"])
us_adult_income["Income"] = us_adult_income["Income"].cat.codes
us_adult_income["Income"] = 1 - us_adult_income["Income"]

us_adult_income.head()

,Age,Workclass,Fnlwgt,Education,EducationNum,MaritalStatus,Occupation,Relationship,Race,Gender,CapitalGain,CapitalLoss,HoursPerWeek,Country,Income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,1
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,1
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,1
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,1
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,1


In [9]:
adult2 = us_adult_income.sample(frac=0.05, random_state=42)
adult2.shape

(1628, 15)

In [11]:
adult25 = adult2.copy()
adult25["Workclass"] = pd.Categorical(adult25["Workclass"])
adult25["Workclass"] = adult25["Workclass"].cat.codes

adult25["Education"] = pd.Categorical(adult25["Education"])
adult25["Education"] = adult25["Education"].cat.codes

adult25["MaritalStatus"] = pd.Categorical(adult25["MaritalStatus"])
adult25["MaritalStatus"] = adult25["MaritalStatus"].cat.codes

adult25["Occupation"] = pd.Categorical(adult25["Occupation"])
adult25["Occupation"] = adult25["Occupation"].cat.codes

adult25["Relationship"] = pd.Categorical(adult25["Relationship"])
adult25["Relationship"] = adult25["Relationship"].cat.codes

adult25["Race"] = pd.Categorical(adult25["Race"])
adult25["Race"] = adult25["Race"].cat.codes

adult25["Gender"] = pd.Categorical(adult25["Gender"])
adult25["Gender"] = adult25["Gender"].cat.codes

adult25["Country"] = pd.Categorical(adult25["Country"])
adult25["Country"] = adult25["Country"].cat.codes

In [12]:
adult23 = adult25[["Age", "Education", "Occupation", "Gender", "Income"]]
adult23

,Age,Education,Occupation,Gender,Income
14160,27,15,0,0,1
27048,45,11,3,0,1
28868,29,9,3,1,0
5667,30,9,6,0,1
7827,29,15,2,1,1
...,...,...,...,...,...
17491,19,11,11,0,1
18954,50,8,9,1,1
15112,44,11,2,1,1
5477,19,15,9,1,1


In [31]:
adult23 = adult23[adult23["Occupation"] != 1]
adult23 = adult23[adult23["Education"] != 13]


In [32]:
adult23.shape

(1625, 5)

In [ ]:
D= np.genfromtxt('../datafiles/iris_train.txt', dtype=str)
np.random.shuffle(D)

D

In [ ]:
D4 = D2.astype(str)
D4

In [41]:
len(D2)

326

In [97]:
class_values = [str(y) for y in list(set([x[len(x)-1] for x in D2]))]
attribute_indexes = [k for k,v in A.items()]

In [33]:
D2 = adult23.to_numpy()
D2 = D2.astype(str)
np.random.shuffle(D2)

In [ ]:
#D2= np.genfromtxt('../datafiles/iris_train.txt', dtype=str)

k=len(D2)

# attribute_name=['sepal length', 'sepal width', 'petal length', 'petal width']

attribute_name = ["Age", "Education", "Occupation", "Gender", "CapitalLoss"]

A={name: [i] for i, name in enumerate(attribute_name)}
# feature_discrete = {name: False for name in attribute_name}
feature_discrete={name: False if isinstance(adult2[name].iloc[0], np.integer) else True for name in attribute_name}

#conf={'A': A, 
#      'feature_discrete': feature_discrete, 
#      'treeType':'IGGR'}

# tree = Tree_DPDT(

# attribute_indexes, 
# attribute_values, 
# root, 
# class_values, 
# IGGR
# dataset_size, 
# epsilon_per_tree
# 
# ) 

root=None

dt=Tree_DPDT(
    attribute_indexes, 
    A, 
    root, # ?,
    class_values,
    feature_discrete, 
    'IGGR', 
    len(D2), 
    0.2)

dt.train(D2[:len(D2)//2])
print(dt.tree)
print(dt.eval(D2[len(D2)//2:]))
dt.prune(D2[:len(D2)//2], D2[len(D2)//2:])
print(dt.tree)
print(dt.eval(D2[len(D2)//2:]))


In [214]:
class simple_node:
    _parent_node = None
    _split_value_from_parent = None
    _splitting_attribute = None
    _level = None
    _id = None
    _children = None
    _class_counts = None
    _noisy_class_counts = None
    _signal_to_noise = 0.0

    def __init__(self, parent_node, split_value_from_parent, splitting_attribute, tree_level, id, children):
        self._parent_node = parent_node
        self._split_value_from_parent = split_value_from_parent
        self._splitting_attribute = splitting_attribute
        self._level = tree_level
        self._id = id
        self._children = children
        self._class_counts = defaultdict(int)
        self._noisy_class_counts = None
        self._signal_to_noise = 0.0

    def add_child(self, child_node):
        self._children.append(child_node)

    def remove_child(self, child_node):
        value = child_node._split_value_from_parent
        if value not in [x._split_value_from_parent for x in self._children]:
            return 0
        else:
            new_children = [x for x in self._children if x._split_value_from_parent != value]
            self._children = new_children
            return 1


    def increment_class_count(self, class_value):
        self._class_counts[str(class_value)] += 1


    def make_noisy_class_counts(self, epsilon, class_values):
        ''' Add noise to the class counts of this leaf. Not performed on parents. '''
        if not self._noisy_class_counts and not self._children: # to make sure this code is only run once per leaf
            counts = {}
            for val in class_values:
                if val in self._class_counts:
                    # Laplace noise added to THE CLASS COUNTS
                    counts[val] = max( 0, int(self._class_counts[val] + np.random.laplace(scale=float(1./epsilon))) )
                else: # original count was 0
                    counts[val] = max( 0, int(np.random.laplace(scale=float(1./epsilon))) )
            self._noisy_class_counts = counts

            # what I'm assuming is equation (10)
            self._signal_to_noise = (epsilon * 1.0 * sum([v for k,v in counts.items()])) / (math.sqrt(2*1.0) * len(counts)) # enS / (sqrt(2n) * num_classes)
            #print("self._signal_to_noise = {}".format(self._signal_to_noise))
            return 0
        else:
            return 1 # we're summing the redundant calls


    def add_childrens_class_counts(self, epsilon):
        ''' Add up the class counts of all leaf nodes that trace back to this parent node. '''
        if not self._noisy_class_counts and self._children:
            counts, num_leafs = self._find_leafs_and_count(self, defaultdict(int), 0)
            self._noisy_class_counts = counts
            self._signal_to_noise =  (epsilon * num_leafs * sum([v for k,v in counts.items()])) / (math.sqrt(2*num_leafs) * len(counts)) # enS / (sqrt(2n) * num_classes)
            return 0
        else:
            return 1 # we're summing the redundant calls
                    
    def _find_leafs_and_count(self, node, counts, num_leafs):
        if node._children:
            for child in node._children:
                counts, num_leafs = self._find_leafs_and_count(child, counts, num_leafs)
        else:
            num_leafs += 1
            for k,v in node._noisy_class_counts.items():
                counts[k] += v
        return counts, num_leafs

In [6]:
from collections import Counter, defaultdict
import random
import numpy as np
import math
import pandas as pd 
import numpy as np 

In [164]:
# currently includes methods as:
        # _init_  : _trees
        # get_domains
        # reduce trees
        # evaluate_accuracy_with_voting

class DPRF_Forest:  
    def __init__(self, 
                 training_nonprocessed,
                 training_data, # 2D list of the training data where the columns are the attributes, and the first column is the class attribute
                 # test_data, # T
                 epsilon, # epsilon, Budget, B, the total privacy budget
                 f, # n of attributes to be split used by each dctree f
                 max_depth # max tree depth 
                 ):

        self._trees = []


        ''' Some initialization '''
        # Attribute set F
        attribute_values = self.get_domains(training_data)

        # Class values set
        class_values = [str(y) for y in list(set([x[len(x)-1] for x in training_data]))]
        attribute_indexes = [int(k) for k,v in attribute_values.items()]


        #print(attribute_values)
        #print("------------")
        #print(class_values)
        #print("------------")
        #print(attribute_indexes)
        #print("-----------")

        
        # |D|
        dataset_size = len(training_data)


        valid_attribute_sizes = [[int(k),len(v)] for k,v in attribute_values.items()]

        print("valid attribute sizes")
        print(valid_attribute_sizes)

        # Number of trees t
        num_trees = len(attribute_indexes)

        print("Number of trees: ", num_trees)

        average_domain = np.mean( [x[1] for x in valid_attribute_sizes] )

        # e = B / t
        epsilon_per_tree = epsilon / float(num_trees)

        ''' Calculating minimum support threshold '''
        # estimated_support_min_depth = dataset_size / (average_domain**2) # a large number
        # estimated_support_max_depth = dataset_size / (average_domain ** (len(attribute_indexes)/2)) # max tree depth is k/2 # a small number
        # epsilon_per_tree = epsilon / float(num_trees)
        # min_support = len(class_values) * math.sqrt(2) * (1/epsilon_per_tree) * SNR_MULTIPLIER # the minimum support in order for S>N


        # while estimated_support_max_depth > min_support: # then we can have more trees
        #    num_trees += 1
        #    epsilon_per_tree = epsilon / float(num_trees)
        #    min_support = len(class_values) * math.sqrt(2) * (1/epsilon_per_tree) * SNR_MULTIPLIER


        # while min_support > estimated_support_min_depth and num_trees>1: # then we need to have less trees
        #    num_trees, valid_attribute_sizes, estimated_support_min_depth = self.reduce_trees(num_trees, valid_attribute_sizes, dataset_size)
        #    epsilon_per_tree = epsilon / float(num_trees)
        #    min_support = len(class_values) * math.sqrt(2) * (1/epsilon_per_tree) * SNR_MULTIPLIER


        print("NUM TREES = {} & EPSILON PER TREE = {}".format(num_trees, epsilon_per_tree))

        # ? 
        root_attributes = []

       
        for a in attribute_indexes:
            if a in [x[0] for x in valid_attribute_sizes]:
                root_attributes.append(a) # OR: [index, support, gini]

        print("root attributes")
        print(root_attributes)

        # for treeId = 1,2 ... t do:
        for t in range(num_trees):

            if not root_attributes:
                root_attributes = attribute_indexes[:]

            # randomly extract |D| samples from D w/ a bagging algo?
            root = random.choice(root_attributes)
            root_attributes.remove(root)



            # TREE BUILDTREE(D, A, C, eps, f, Dm)
            # D :

            # attribute_indexes, - A_ind
            # attribute_values, A - not exactly the same after all! 
            # root, 
            # class_values, 
            # feature_discrete
            # treetype 
            # dataset_size, 
            # epsilon_per_tree

            A={name: [i] for i, name in enumerate(adult23.columns[:-1])}

            # {'Age': False, 'Education': True, 'Occupation': True, 'Gender': True, 'CapitalLoss': False}
            feature_discrete={name: False if isinstance(training_nonprocessed[name].iloc[0], np.integer) else True for name in adult23.columns[:-1]}

            print("feature discrete")
            print(feature_discrete)

            print("---------------")
            print("A: ")
            print(A)
            print("---------------")
            print("attribute values: ")
            print(attribute_values)
            print("---------------")
            print("---------------")

            tree = Tree_DPDT(attribute_indexes, 
                            A, # original style A
                            attribute_values, # new style attribute vals
                            root, 
                            class_values, 
                            feature_discrete, 
                            'IGGR',  # treetype
                            dataset_size, 
                            epsilon_per_tree) 

            tree.train(training_data[:len(training_data)//2])
           
            # TO EVOLVE HERE 

            # attribute_name = ["Age", "Education", "Occupation", "Gender", "CapitalLoss"]

            #num_unclassified = tree.filter_training_data_and_count(training_data, epsilon_per_tree, class_values)
            
            #if num_unclassified > 0:
            #    print("number of unclassified records = {}".format(num_unclassified))
            

            print("currently pruning! ")
            # D2[:len(D2)//2], D2[len(D2)//2:]
            tree.prune(training_data[:len(training_data)//2], training_data[len(training_data)//2:])

            # FOREST = FOREST U TREE
            self._trees.append(tree)

            print("succesfully ended building tree dictionary")


    def get_domains(self, data):
        attr_domains = {}
        transData = np.transpose(data)
        for i in range(0,len(data[0]) - 1):
            attr_domains[str(i)] = [str(x) for x in set(transData[i])]
            print("original domain length of categorical attribute {}: {}".format(i, len(attr_domains[str(i)])))
        return attr_domains


    def reduce_trees(self, num_trees, valid_attribute_sizes, dataset_size):
        num_trees -= 1
        largest_attribute = sorted(valid_attribute_sizes, key=lambda x:x[1], reverse=True)[0]
        #print("Removing att{} with domain size {}".format(largest_attribute[0], largest_attribute[1])) 
        new_valids = [ x for x in valid_attribute_sizes if x[0] != largest_attribute[0] ]
        average_domain = np.mean( [x[1] for x in valid_attribute_sizes] ) # average with all attributes
        narrowed_average_domain = np.mean([x[1] for x in valid_attribute_sizes if x[0] in [y[0] for y in new_valids] ]) # average without the big attributes
        estimated_support_squared = dataset_size / (average_domain * narrowed_average_domain)
        return num_trees, new_valids, estimated_support_squared


    # GET MAJORITY LABELS (Forest, T, C) equivalent from pseudocode
    def evaluate_accuracy_with_voting(self, records, class_index=4):

        ''' Calculate the Prediction Accuracy of the Forest. '''
        actual_labels = [x[class_index] for x in records]

        predicted_labels = []
        leafs_not_used = 0
        count_of_averages_used = 0

        h = []
        class_counts = defaultdict(list)

        # FOR EACH RECORD X IN DATASET DO
        # Strange for we need to access individual predictions here...
        for rec in records:

            class_value_fractions = defaultdict(list)

            # FOR EACH TREE DO
            for tree in self._trees:

                # GET PREDICTED CLASSIFICATION RESULT for a single record
                # node, leaf_not_used = tree._classify(tree._root_node, rec)

                print("tree")
                print(tree)

                node = tree.pred(tree, np.array([rec]))


                print("precited ", node , " for record ", rec)

                h.append(*node)

                print("result list")
                print(h)

                # noisy_class_counts = node._noisy_class_counts
                # defaultdict(<class 'int'>, {'0': 2, '1': 33})
                unique, counts = np.unique(h, return_counts=True)
                class_counts = dict(zip(unique, h))

                print("class counts")
                print(class_counts)

                leafs_not_used += leaf_not_used

                support = float(sum([v for k,v in noisy_class_counts.items()]))

                for k,v in noisy_class_counts.items():
                    class_value_fractions[k].append(v/support)
                    
            best_confidences = {}

            for k,lis in class_value_fractions.items():
               best_confidences[k] = max(lis)
            best = [None, 0.0]

            for k,class_best in best_confidences.items():
                if class_best > best[1]:
                    best = [k, class_best]
            average_used = False


            for k, class_best in best_confidences.items():

                if class_best == best[1] and k != best[0]:

                    average_used = True
                    #print("original best: {} vs. contender: {}".format(best, (k,class_best)))

                    orig_average = np.mean(class_value_fractions[best[0]])

                    contender_average = np.mean(class_value_fractions[k])
                    #print("original average: {} vs. contender: {}".format(orig_average, contender_average))

                    if contender_average > orig_average:
                        best = [k, class_best]

            count_of_averages_used += 1 if average_used else 0
            predicted_labels.append(int(best[0]))

        counts = Counter([x == y for x, y in zip(predicted_labels, actual_labels)])
        
        # acc = float(counts[True]) / len(records)
        
        return float(counts[True]) / len(records)
        


In [65]:
len(D2)

1625

In [165]:

# Input:
    # Training set D 
    # Privacy budget B (epsilon)

    # Test Set T (?)  (Maybe?)

    # Number of attributes to be split used by each dec tree f 
    # Max tree depth d_m

B = 0.2 # epsilon
f = 3
dm = 5

# predict print

diff_priv_forest = DPRF_Forest(adult2, D2[400:], B, f, dm)
#num_trees = len(our_diff_priv_forest._trees)
#av_prunings = np.mean([x._prunings for x in our_diff_priv_forest._trees])
#av_tree_size = np.mean([x._id-x._prunings+1 for x in our_diff_priv_forest._trees])
#accuracy, leafs_not_most_confident, votes_requiring_average = diff_priv_forest.evaluate_accuracy_with_voting(adult23, class_index=0) # confidence count includes all trees (before voting)
#print("accuracy = {}".format(accuracy))

original domain length of categorical attribute 0: 63
original domain length of categorical attribute 1: 15
original domain length of categorical attribute 2: 14
original domain length of categorical attribute 3: 2
valid attribute sizes
[[0, 63], [1, 15], [2, 14], [3, 2]]
Number of trees:  4
NUM TREES = 4 & EPSILON PER TREE = 0.05
root attributes
[0, 1, 2, 3]
feature discrete
{'Age': False, 'Education': True, 'Occupation': True, 'Gender': True}
---------------
A: 
{'Age': [0], 'Education': [1], 'Occupation': [2], 'Gender': [3]}
---------------
attribute values: 
{'0': ['41', '31', '29', '47', '51', '30', '70', '79', '66', '19', '48', '73', '64', '52', '60', '83', '90', '67', '39', '42', '33', '43', '46', '20', '58', '22', '49', '81', '59', '38', '23', '62', '55', '40', '36', '71', '53', '57', '72', '45', '65', '28', '37', '44', '35', '21', '69', '50', '56', '34', '24', '63', '32', '54', '27', '68', '26', '61', '74', '80', '17', '25', '18'], '1': ['5', '8', '11', '0', '6', '2', '10', '1

In [121]:
g = D2[400:][:,-1]
unique, counts = np.unique(g, return_counts=True)
print(np.asarray((unique, counts)).T)

[['0' '295']
 ['1' '930']]


In [155]:
D2[400:][:,0:5]

array([['29', '11', '7', '0', '1'],
       ['19', '2', '5', '1', '1'],
       ['59', '15', '0', '0', '1'],
       ...,
       ['29', '4', '7', '1', '1'],
       ['45', '14', '9', '1', '0'],
       ['21', '15', '-1', '1', '1']], dtype='<U21')

In [166]:
accuracy = diff_priv_forest.evaluate_accuracy_with_voting(D2[400:][:,0:5], class_index=4) # confidence count includes all trees (before voting)
print("accuracy = {}".format(accuracy))

tree
precited  [['1']]  for record  ['29' '11' '7' '0' '1']
result list
[array(['1'], dtype='<U1')]
class counts
{'1': array(['1'], dtype='<U1')}


NameError: name 'leaf_not_used' is not defined

In [134]:
accuracy

[array([['1']], dtype='<U1')]

In [ ]:
# Here the classification result set is the left column - 
D2[100:]